In [1]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [3]:
client.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment_tracking/mlruns/1', creation_time=1721214263552, experiment_id='1', last_update_time=1721214263552, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1721059441814, experiment_id='0', last_update_time=1721059441814, lifecycle_stage='active', name='Default', tags={}>]

In [4]:
client.create_experiment(name="my-experiment")

'2'

In [5]:
from mlflow.entities import ViewType

In [9]:
runs = client.search_runs(
    experiment_ids="1",
    filter_string="metrics.rmse < 6.3110",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [10]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 3545158565b946f399a4d65886d3a71f, rmse: 6.3100
run id: 4aa164de3de84962b3ab518d4fcab56a, rmse: 6.3100
run id: 4581607955cf4495a57d0b2b3a475d81, rmse: 6.3100


In [11]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [12]:
model_uri = "runs:/3545158565b946f399a4d65886d3a71f/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-xgboost")

Registered model 'nyc-taxi-xgboost' already exists. Creating a new version of this model...
Created version '2' of model 'nyc-taxi-xgboost'.


<ModelVersion: aliases=[], creation_timestamp=1721401744166, current_stage='None', description=None, last_updated_timestamp=1721401744166, name='nyc-taxi-xgboost', run_id='3545158565b946f399a4d65886d3a71f', run_link=None, source='/workspaces/mlops-zoomcamp/02-experiment_tracking/mlruns/1/3545158565b946f399a4d65886d3a71f/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [15]:
model_name = "nyc-taxi-xgboost"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.tags}")

version: 2, stage: {}


/tmp/ipykernel_13251/2240107009.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [ ]:
from sklearn.metrics import mean_squared_error
import pandas as pd

def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df["duration"] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df

def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + "_" + df['DOLocationID']
    categorical = ["PU_DO"]
    numerical = ["trip_distance"]
    train_dicts = df[categorical + numerical].to_dict(orient="records")
    return dv.transform(train_dicts)

def test_model(stage, X_test, y_test, name="nyc-taxi_xgboost"):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")